<a href="https://colab.research.google.com/github/Shoaib003/Portfolio_project_shoaib/blob/main/PD_Value_%26_Performance_controller_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Phase 1 - Data Generation + Schema Building

1. Generating realistic data with python
2. save it as clean CSVs
3. Build the Sql schema + views

## Generating realistic Financial & Operational Data

### Assumptions

- Monthly Data
- 3 advisory departments
- Revenue driven by utilization x day rate
- Costs scale with revenue but include randomness
- Budget =/ Actual

In [1]:
import pandas as pd
import numpy as np

In [2]:
# generating random data

np.random.seed(42)

In [3]:
# --- Parameters ---

months = pd.date_range(start = '2023-01-01', periods=24, freq='M')
departments = {
    1: "Digital Transformation",
    2: "Public Finance",
    3: "Infrastructure Advisory"
}

avg_daily_rate = {
    1: 1100,
    2: 950,
    3: 1000
}

available_days_per_month = 20

/tmp/ipython-input-1633503940.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  months = pd.date_range(start = '2023-01-01', periods=24, freq='M')


In [5]:
# --- Data Containers ---
financial_rows = []
operations_rows = []

# --- Data Generation ---
for dept_id, dept_name in departments.items():
  for month in months:
    utilization = np.clip(np.random.normal(0.78, 0.05), 0.65, 0.9)
    billable_days = utilization * available_days_per_month

    revenue = billable_days * avg_daily_rate[dept_id] * np.random.uniform(0.95, 1.05)

    personnel_cost = revenue * np.random.uniform(0.45, 0.55)
    overhead_cost = revenue * np.random.uniform(0.12, 0.18)
    external_cost = revenue * np.random.uniform(0.05, 0.10)

    budget_revenue = revenue * np.random.uniform(0.97, 1.03)
    budget_cost = (personnel_cost + overhead_cost + external_cost) * np.random.uniform(0.97, 1.03)

    financial_rows.append({
        "date": month,
        "department_id": dept_id,
        "revenue": round(revenue,2),
        "personnel_cost": round(personnel_cost,2),
        "overhead_cost": round(overhead_cost,2),
        "external_cost": round(external_cost,2),
        "budget_revenue": round(budget_revenue,2),
        "budget_cost": round(budget_cost,2)
    })

    operations_rows.append({
        "date": month,
        "department_id": dept_id,
        "billable_days": round(billable_days,1),
        "available_days": available_days_per_month,
        "utilization_rate": round(utilization,3)
    })

# --- Create DataFrames ---
df_financials = pd.DataFrame(financial_rows)
df_operations = pd.DataFrame(operations_rows)

df_departments = pd.DataFrame({
    "department_id": list(departments.keys()),
    "department_name": list(departments.values())
})

In [6]:
# --- Save to CSV ---
df_financials.to_csv("fact_financials.csv", index=False)
df_operations.to_csv("fact_operations.csv", index=False)
df_departments.to_csv("dim_departments.csv", index=False)

In [7]:
df_financials.head()

,date,department_id,revenue,personnel_cost,overhead_cost,external_cost,budget_revenue,budget_cost
0,2023-01-31,1,16888.91,8061.76,2083.80,1574.66,17205.98,12071.62
1,2023-02-28,1,19441.45,9828.49,3229.99,1890.45,19849.31,14722.32
2,2023-03-31,1,16711.45,8533.16,2234.63,1396.83,16829.89,12061.09
3,2023-04-30,1,16346.30,8453.61,2471.86,1448.54,16366.08,12635.49
4,2023-05-31,1,20158.54,9884.71,2581.12,1036.94,20467.13,13600.23


In [8]:
df_operations.head()

,date,department_id,billable_days,available_days,utilization_rate
0,2023-01-31,1,15.5,20,0.774
1,2023-02-28,1,16.8,20,0.842
2,2023-03-31,1,14.5,20,0.727
3,2023-04-30,1,15.5,20,0.773
4,2023-05-31,1,17.7,20,0.883


In [9]:
df_departments

,department_id,department_name
0,1,Digital Transformation
1,2,Public Finance
2,3,Infrastructure Advisory


## Creating the SQL Schema (SQLite)

In [36]:
import sqlite3

conn = sqlite3.connect("controlling_portfolio.db")
cursor = conn.cursor()

In [37]:
cursor.execute("""
Create TABLE IF NOT EXISTS dim_department (
  department_id INTEGER PRIMARY KEY,
  department_name TEXT
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS fact_financials (
  date TEXT,
  department_id INTEGER,
  revenue REAL,
  personnel_cost REAL,
  overhead_cost REAL,
  external_cost REAL,
  budget_revenue REAL,
  budget_cost REAL,
  PRIMARY KEY (date, department_id),
  FOREIGN KEY (department_id) REFERENCES dim_department(department_id)

);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS fact_operations (
  date TEXT,
  department_id INTEGER,
  billable_days REAL,
  available_days INTEGER,
  utilization_rate REAL,
  PRIMARY KEY (date, department_id)

);
""")

conn.commit()

## Load Data into SQLite

### Insert DataFrames into SQL

In [38]:
df_departments.to_sql("dim_department", conn, if_exists="append", index=False)

3

In [39]:
df_financials.to_sql("fact_financials", conn, if_exists="append", index=False)

72

In [40]:
df_operations.to_sql("fact_operations", conn, if_exists="append", index=False)

72

### Test SQL Queries

In [41]:
query1 = """
SELECT *
FROM dim_department;
"""
pd.read_sql(query1, conn)

,department_id,department_name
0,1,Digital Transformation
1,2,Public Finance
2,3,Infrastructure Advisory


In [42]:
query = """
SELECT
  f.date,
  d.department_name,
  f.revenue,
  (f.revenue - (f.personnel_cost + f.overhead_cost + f.external_cost)) AS profit,
  (f.revenue - f.budget_revenue) AS revenue_variance
FROM fact_financials f
JOIN dim_department d
ON f.department_id = d.department_id
LIMIT 10;
"""

pd.read_sql(query, conn)

,date,department_name,revenue,profit,revenue_variance
0,2023-01-31 00:00:00,Digital Transformation,16888.91,5168.69,-317.07
1,2023-02-28 00:00:00,Digital Transformation,19441.45,4492.52,-407.86
2,2023-03-31 00:00:00,Digital Transformation,16711.45,4546.83,-118.44
3,2023-04-30 00:00:00,Digital Transformation,16346.30,3972.29,-19.78
4,2023-05-31 00:00:00,Digital Transformation,20158.54,6655.77,-308.59
5,2023-06-30 00:00:00,Digital Transformation,19480.48,6814.16,174.64
6,2023-07-31 00:00:00,Digital Transformation,16499.35,4539.83,102.42
7,2023-08-31 00:00:00,Digital Transformation,15457.38,3553.56,7.31
8,2023-09-30 00:00:00,Digital Transformation,17569.49,5220.13,447.11
9,2023-10-31 00:00:00,Digital Transformation,16487.10,5349.02,314.69


## CREATE CSV file for TABLEAU

In [43]:
df_tableau = pd.read_sql("""
SELECT
  f.date,
  d.department_name,
  f.revenue,
  f.budget_revenue,
  (f.revenue - (f.personnel_cost + f.overhead_cost + f.external_cost)) AS profit,
  o.utilization_rate
FROM fact_financials f
JOIN fact_operations o
ON f.date = o.date AND f.department_id = o.department_id
JOIN dim_department d
ON f.department_id = d.department_id
""", conn)

In [44]:
df_tableau.to_csv("tableau_dataset.csv", index=False)

# SUMMARY

I used Python on Google Colab to generate driver-based financial data, built a star-schema in SQLite, and used SQL for KPI and variance analysis before exporting datasets to Tableau.

## Advanced SQL Views + Rolling YTD Logic

### Opening the existing SQLite connection

In [45]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("controlling_portfolio.db")

### Monthly Performance View (Profit & Variance)

- Are we profitable?
- Are we above/below budget?
- Which department performs better?

In [46]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_monthly_performance AS
SELECT
  f.date,
  d.department_name,
  f.revenue,
  (f.personnel_cost + f.overhead_cost + f.external_cost) AS total_cost,
  (f.revenue - (f.personnel_cost + f.overhead_cost + f.external_cost)) AS profit,
  ((f.personnel_cost + f.overhead_cost + f.external_cost) - f.budget_cost) AS cost_variance

FROM fact_financials f
JOIN dim_department d
ON f.department_id = d.department_id;
""")

In [47]:
pd.read_sql("SELECT * FROM vw_monthly_performance LIMIT 5", conn)

,date,department_name,revenue,total_cost,profit,cost_variance
0,2023-01-31 00:00:00,Digital Transformation,16888.91,11720.22,5168.69,-351.40
1,2023-02-28 00:00:00,Digital Transformation,19441.45,14948.93,4492.52,226.61
2,2023-03-31 00:00:00,Digital Transformation,16711.45,12164.62,4546.83,103.53
3,2023-04-30 00:00:00,Digital Transformation,16346.30,12374.01,3972.29,-261.48
4,2023-05-31 00:00:00,Digital Transformation,20158.54,13502.77,6655.77,-97.46


### KPI View (Value & Performance Focus)

- How efficient are we?
- Is utilization driving profitability?
- How strong is EBIT margin?

In [48]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_kpis AS
SELECT
  f.date,
  d.department_name,
  f.revenue,
  ROUND(
    (f.revenue - (f.personnel_cost + f.overhead_cost + f.external_cost))
    / f.revenue, 3
  ) AS ebit_margin,
  o.utilization_rate,
  ROUND(f.revenue / o.billable_days, 2) AS revenue_per_day
FROM fact_financials f
JOIN fact_operations o
ON f.date = o.date AND f.department_id = o.department_id
JOIN dim_department d
ON f.department_id = d.department_id;
""")

In [49]:
pd.read_sql("SELECT * FROM vw_kpis LIMIT 5", conn)

,date,department_name,revenue,ebit_margin,utilization_rate,revenue_per_day
0,2023-01-31 00:00:00,Digital Transformation,16888.91,0.306,0.774,1089.61
1,2023-02-28 00:00:00,Digital Transformation,19441.45,0.231,0.842,1157.23
2,2023-03-31 00:00:00,Digital Transformation,16711.45,0.272,0.727,1152.51
3,2023-04-30 00:00:00,Digital Transformation,16346.30,0.243,0.773,1054.60
4,2023-05-31 00:00:00,Digital Transformation,20158.54,0.330,0.883,1138.90


### Rolling 12-Month Revenue

- removes seasonality
- used for trends & performance steering
- very common in board reporting

In [50]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_rolling_12m_revenue AS
SELECT
  date,
  department_name,
  SUM(revenue) OVER (
      PARTITION BY department_name
      ORDER BY date
      ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
  ) AS rolling_12m_revenue
FROM vw_monthly_performance;
""")

In [51]:
pd.read_sql("""
SELECT *
FROM vw_rolling_12m_revenue
WHERE department_name = 'Digital Transformation'
LIMIT 15
""", conn)

,date,department_name,rolling_12m_revenue
0,2023-01-31 00:00:00,Digital Transformation,16888.91
1,2023-02-28 00:00:00,Digital Transformation,36330.36
2,2023-03-31 00:00:00,Digital Transformation,53041.81
3,2023-04-30 00:00:00,Digital Transformation,69388.11
4,2023-05-31 00:00:00,Digital Transformation,89546.65
5,2023-06-30 00:00:00,Digital Transformation,109027.13
6,2023-07-31 00:00:00,Digital Transformation,125526.48
7,2023-08-31 00:00:00,Digital Transformation,140983.86
8,2023-09-30 00:00:00,Digital Transformation,158553.35
9,2023-10-31 00:00:00,Digital Transformation,175040.45


### YTD (Year-To-Date) Performance View

- Budget tracking
- Management steering
- Mid-year correction decisions

In [52]:
conn.execute("""
CREATE VIEW IF NOT EXISTS vw_ytd_performance AS
SELECT
  date,
  department_name,
  SUM(revenue) OVER (
  PARTITION BY department_name, strftime('%Y', date)
  ORDER BY date
  ) AS ytd_revenue,
  SUM(profit) OVER (
  PARTITION BY department_name, strftime('%Y', date)
  ORDER BY date
  ) AS ytd_profit
FROM vw_monthly_performance;
""")

In [54]:
pd.read_sql("""
SELECT *
FROM vw_ytd_performance
WHERE department_name = 'Public Finance'
LIMIT 12
""", conn)

,date,department_name,ytd_revenue,ytd_profit
0,2023-01-31 00:00:00,Public Finance,14748.27,4255.72
1,2023-02-28 00:00:00,Public Finance,28481.05,8557.63
2,2023-03-31 00:00:00,Public Finance,44501.27,12907.91
3,2023-04-30 00:00:00,Public Finance,58702.54,16309.06
4,2023-05-31 00:00:00,Public Finance,73715.78,20027.34
5,2023-06-30 00:00:00,Public Finance,87895.29,23323.75
6,2023-07-31 00:00:00,Public Finance,103759.30,28074.64
7,2023-08-31 00:00:00,Public Finance,117805.15,31443.27
8,2023-09-30 00:00:00,Public Finance,130736.56,34504.40
9,2023-10-31 00:00:00,Public Finance,146089.06,39059.80


## Summary

I created SQL views for montly performance, KPI analysis, rolling 12-month revenue, and YTD tracking to support mananagement reporting and BI dashboards.

# Python Analysis & Sensitivity Modeling

- Analyse profit & utilization drivers
- Quantify risk and sensitivity
- Produce management-ready insights
- Generate datasets later used in Excel & Tableau

### Load SQL Views into Python

In [55]:
import pandas as pd

import sqlite3

conn = sqlite3.connect("controlling_portfolio.db")

In [56]:
df_perf = pd.read_sql("SELECT * FROM vw_monthly_performance", conn)

In [57]:
df_kpi = pd.read_sql("SELECT * FROM vw_kpis", conn)

In [58]:
df_rolling = pd.read_sql("SELECT * FROM vw_rolling_12m_revenue", conn)

In [59]:
df_perf.head()

,date,department_name,revenue,total_cost,profit,cost_variance
0,2023-01-31 00:00:00,Digital Transformation,16888.91,11720.22,5168.69,-351.40
1,2023-02-28 00:00:00,Digital Transformation,19441.45,14948.93,4492.52,226.61
2,2023-03-31 00:00:00,Digital Transformation,16711.45,12164.62,4546.83,103.53
3,2023-04-30 00:00:00,Digital Transformation,16346.30,12374.01,3972.29,-261.48
4,2023-05-31 00:00:00,Digital Transformation,20158.54,13502.77,6655.77,-97.46


In [60]:
df_kpi.head()

,date,department_name,revenue,ebit_margin,utilization_rate,revenue_per_day
0,2023-01-31 00:00:00,Digital Transformation,16888.91,0.306,0.774,1089.61
1,2023-02-28 00:00:00,Digital Transformation,19441.45,0.231,0.842,1157.23
2,2023-03-31 00:00:00,Digital Transformation,16711.45,0.272,0.727,1152.51
3,2023-04-30 00:00:00,Digital Transformation,16346.30,0.243,0.773,1054.60
4,2023-05-31 00:00:00,Digital Transformation,20158.54,0.330,0.883,1138.90


In [61]:
df_rolling.head()

,date,department_name,rolling_12m_revenue
0,2023-01-31 00:00:00,Digital Transformation,16888.91
1,2023-02-28 00:00:00,Digital Transformation,36330.36
2,2023-03-31 00:00:00,Digital Transformation,53041.81
3,2023-04-30 00:00:00,Digital Transformation,69388.11
4,2023-05-31 00:00:00,Digital Transformation,89546.65


### Core Performance Analysis

#### Profit Trend Over Time

- Are we improving or deteriorating?

In [62]:
df_profit_trend = (
    df_perf.groupby("date", as_index=False)["profit"].sum()
)

In [64]:
df_profit_trend.tail()    # this shows overall financial trajectory across all departments

,date,profit
19,2024-08-31 00:00:00,13895.16
20,2024-09-30 00:00:00,10861.29
21,2024-10-31 00:00:00,12125.76
22,2024-11-30 00:00:00,12235.73
23,2024-12-31 00:00:00,13175.59


#### Department Contribution Analysis
- Which unit creates value?

In [65]:
df_dept_summary = (
    df_perf.groupby("department_name").agg(
        total_revenue = ("revenue", "sum"),
        total_profit = ("profit", "sum"),
        avg_monthly_profits = ("profit", "mean")
    )
    .reset_index()
)

In [66]:
df_dept_summary

,department_name,total_revenue,total_profit,avg_monthly_profits
0,Digital Transformation,409378.16,115759.84,4823.326667
1,Infrastructure Advisory,376135.45,104356.20,4348.175000
2,Public Finance,356755.22,95482.46,3978.435833


#### Utilization vs Profit (Key Driver)

In [67]:
# merging KPI and Profit Data

df_driver = pd.merge(
    df_kpi,
    df_perf[["date", "department_name", "profit"]],
    on = ["date", "department_name"]
)

#### Correlation Analysis

In [68]:
df_driver[["utilization_rate", "profit"]].corr()

,utilization_rate,profit
utilization_rate,1.000000,0.523599
profit,0.523599,1.000000


#### Talking point

Profitability is strongly driven by utilization, confirming staffing efficiency as a key value lever.

#### Sensitivity Analysis

What happens if utilization drops by 5% or 10%?

In [70]:
# Creating a Sensitivity Function

def simulate_utilization_change(df, utilization_change):
  simulated = df.copy()
  simulated["simulated_revenue"] = simulated["revenue"] * (1 + utilization_change)
  simulated["simulated_profit"] = (simulated["simulated_revenue"] - (simulated["revenue"] - simulated["profit"]))

  return simulated

In [72]:
# Running Scenarios

scenario_base = simulate_utilization_change(df_perf, 0)
scenario_minus_5 = simulate_utilization_change(df_perf, -0.05)
scenario_minus_10 = simulate_utilization_change(df_perf, -0.10)

In [74]:
# Comparing the results

def scenario_summary(df, label):
  return pd.DataFrame({
      "scenario": [label],
      "total_revenue": [df["simulated_revenue"].sum()],
      "total_profit": [df["simulated_profit"].sum()]
  })

summary = pd.concat([
    scenario_summary(scenario_base, "Base"),
    scenario_summary(scenario_minus_5, "-5% Utilization"),
    scenario_summary(scenario_minus_10, "-10% Utilization")
])

summary

,scenario,total_revenue,total_profit
0,Base,1.142269e+06,315598.5000
0,-5% Utilization,1.085155e+06,258485.0585
0,-10% Utilization,1.028042e+06,201371.6170


### Insights

- A 5% utilization drop causes a disproportionate EBIT decline
- Fixed costs amplify downside risk
- Staffing optimization has higher impact than cost cutting

In [75]:
# Export Analysis Results

summary.to_csv("utilization_sensitivity_summary.csv", index = False)
df_driver.to_csv("driver_analysis_dataset.csv", index = False)

I used Python to analyse value drivers, quantify utilization sensitivity, and simulate downside risk scenarios to support management decisions.

## Exporting Actuals from Python

In [82]:
df_actuals_excel = pd.read_sql ("""
SELECT
  f.date,
  d.department_name,
  f.revenue,
  ROUND((f.personnel_cost + f.overhead_cost + f.external_cost),2) AS total_cost,
  ROUND((f.revenue - (f.personnel_cost + f.overhead_cost + f.external_cost)),2) AS profit,
  o.utilization_rate
FROM fact_financials f
JOIN fact_operations o
ON f.date = o.date AND f.department_id = o.department_id
JOIN dim_department d
ON f.department_id = d.department_id
""", conn)

In [84]:
df_actuals_excel.head() # checking

,date,department_name,revenue,total_cost,profit,utilization_rate
0,2023-01-31 00:00:00,Digital Transformation,16888.91,11720.22,5168.69,0.774
1,2023-02-28 00:00:00,Digital Transformation,19441.45,14948.93,4492.52,0.842
2,2023-03-31 00:00:00,Digital Transformation,16711.45,12164.62,4546.83,0.727
3,2023-04-30 00:00:00,Digital Transformation,16346.30,12374.01,3972.29,0.773
4,2023-05-31 00:00:00,Digital Transformation,20158.54,13502.77,6655.77,0.883


In [83]:
df_actuals_excel.to_csv("excel_actuals.csv", index=False)